# Choroid Plexus Segmentation Training via Auto3DSeg

In [1]:
import os
import json
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import glob
import random
import platform
from monai.apps.auto3dseg import AutoRunner
from monai.config import print_config
import importlib
from dataclasses import asdict
from loguru import logger
import sys

from reload_recursive import reload_recursive

import mri_data
import monai_training

print_config()

/home/srs-9/.virtualenvs/monai/lib/python3.12/site-packages/ignite/handlers/checkpoint.py:17: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


MONAI version: 1.3.2
Numpy version: 1.26.4
Pytorch version: 2.4.0+cu121
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 59a7211070538586369afd4a01eca0a7fe2e742e
MONAI __file__: /home/<username>/.virtualenvs/monai/lib/python3.12/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.11
ITK version: 5.4.0
Nibabel version: 5.2.1
scikit-image version: 0.24.0
scipy version: 1.14.0
Pillow version: 10.4.0
Tensorboard version: 2.17.0
gdown version: 5.2.0
TorchVision version: 0.19.0+cu121
tqdm version: 4.66.4
lmdb version: 1.5.1
psutil version: 6.0.0
pandas version: 2.2.2
einops version: 0.8.0
transformers version: 4.43.3
mlflow version: 2.15.0
pynrrd version: 1.0.0
clearml version: 1.16.2

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [2]:
reload_recursive(monai_training)
reload_recursive(mri_data)

from mri_data.file_manager import scan_3Tpioneer_bids, filter_first_ses
from monai_training.preprocess import DataSetProcesser
from monai_training import training, preprocess

In [3]:
logger.remove()
logger.add(sys.stderr, level="INFO")

1

In [4]:
load_data = False

In [5]:
hostname = platform.node()
if hostname == "rhinocampus" or hostname == "ryzen9":
    drive_root = Path("/media/smbshare")
else:
    drive_root = Path("/mnt/h")

projects_root = Path("/home/srs-9/Projects")
msmri_home = projects_root / "ms_mri"
training_work_dirs = msmri_home / "training_work_dirs"

dataroot = drive_root / "3Tpioneer_bids"
work_dir_name = "choroid_pineal_pituitary3"
work_dir = training_work_dirs / work_dir_name
modalities = ["flair", "t1"]
labels = ["choroid_t1_flair", "pineal", "pituitary"]

## Prep the database

Get the data and labels organized

In [6]:
if load_data:
    datalist_file = os.path.join(work_dir, "datalist.json")
    with open(datalist_file, 'r') as f:
        datalist = json.load(f)

    dataset = preprocess.parse_datalist(datalist_file, dataroot)

In [9]:
if not load_data:
    dataset, _ = preprocess.load_dataset("dataset.json")

In the original code, they include labels in the test data as well. Also they have a function that checks that there is nonzero number of voxels in the label

In [11]:
#? I don't know why they put labels for the test data. the brats segmentation code didn't.
# train_data = [{'image': path + '/flair.nii.gz', 'label': path + '/flair_chp_mask_qced.nii.gz'} for path in train_exams]
# test_data = [{'image': path + '/flair.nii.gz', 'label': path + '/flair_chp_mask_qced.nii.gz'} for path in test_exams]
if not load_data:
    train_data = []
    test_data = []
    for scan in dataset:
        if scan.cond == 'tr' and scan.has_label:
            train_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})
        elif scan.cond == 'ts' and scan.has_label():
            test_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})


    print(f"Train num total: {len(train_data)}")
    print(f"Test num: {len(test_data)}")

Train num total: 30
Test num: 10


Create and save datalist

In [13]:
if not load_data:
    n_folds = 5
    datalist = {
        "testing": test_data,
        "training": [{"fold": i % n_folds, "image": c["image"], "label": c["label"]} for i,c in enumerate(train_data)]
    }

    if not os.path.isdir(work_dir):
        os.makedirs(work_dir)

    # dataroot_dir = "/mnt/h"
    # if not os.path.isdir(dataroot_dir):
    #     os.makedirs(dataroot_dir)

    datalist_file = os.path.join(work_dir, "datalist.json")
    with open(datalist_file, "w") as f:
        json.dump(datalist, f, indent=4)

Load datalist

In [14]:
datalist['testing']

[{'image': '/media/smbshare/3Tpioneer_bids/sub-ms1109/ses-20180303/flair.t1.nii.gz',
  'label': '/media/smbshare/3Tpioneer_bids/sub-ms1109/ses-20180303/choroid_t1_flair-ED.pineal-CH.pituitary-CH.nii.gz'},
 {'image': '/media/smbshare/3Tpioneer_bids/sub-ms1188/ses-20200720/flair.t1.nii.gz',
  'label': '/media/smbshare/3Tpioneer_bids/sub-ms1188/ses-20200720/choroid_t1_flair-ED.pineal-CH.pituitary-CH.nii.gz'},
 {'image': '/media/smbshare/3Tpioneer_bids/sub-ms1259/ses-20200803/flair.t1.nii.gz',
  'label': '/media/smbshare/3Tpioneer_bids/sub-ms1259/ses-20200803/choroid_t1_flair-ED.pineal-SRS.pituitary-CH.nii.gz'},
 {'image': '/media/smbshare/3Tpioneer_bids/sub-ms1547/ses-20220321/flair.t1.nii.gz',
  'label': '/media/smbshare/3Tpioneer_bids/sub-ms1547/ses-20220321/choroid_t1_flair-ED.pineal-SRS.pituitary-CH.nii.gz'},
 {'image': '/media/smbshare/3Tpioneer_bids/sub-ms2081/ses-20170204/flair.t1.nii.gz',
  'label': '/media/smbshare/3Tpioneer_bids/sub-ms2081/ses-20170204/choroid_t1_flair-ED.pineal

In [15]:
missing_images = []
missing_labels = []
for scan in dataset:
    if not scan.image_path.is_file():
        missing_images.append(scan.image_path)
    if not scan.label_path.is_file():
        missing_images.append(scan.label_path)
print(missing_images)
print(missing_labels)

[]
[]


In [16]:
runner = AutoRunner(
    work_dir=work_dir,
    algos=["swinunetr"],
    input={
        "modality": "MRI",
        "datalist": str(datalist_file),
        "dataroot": str(dataroot),
    },
)

2024-11-08 17:07:52,570 - INFO - AutoRunner using work directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary2
2024-11-08 17:07:52,571 - INFO - Found num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary2/datalist.json.
2024-11-08 17:07:52,572 - INFO - Setting num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary2/datalist.json.
2024-11-08 17:07:52,583 - INFO - Using user defined command running prefix , will override other settings


In [17]:
max_epochs = 100

train_param = {
    "num_epochs_per_validation": 1,
    #"num_images_per_batch": 2,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 1,
}
runner.set_training_params(train_param)

In [18]:
runner.run()

2024-11-08 17:07:58,820 - INFO - Running data analysis...
2024-11-08 17:07:58,821 - INFO - Found 1 GPUs for data analyzing!


100%|██████████| 24/24 [02:31<00:00,  6.31s/it]

2024-11-08 17:10:30,301 - INFO - Data spacing is not completely uniform. MONAI transforms may provide unexpected result
2024-11-08 17:10:30,301 - INFO - Writing data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary2/datastats.yaml.
2024-11-08 17:10:30,307 - INFO - Writing by-case data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary2/datastats_by_case.yaml, this may take a while.


2024-11-08 17:10:30,604 - INFO - BundleGen from https://github.com/Project-MONAI/research-contributions/releases/download/algo_templates/e4cf5a1.tar.gz


algo_templates.tar.gz: 104kB [00:00, 432kB/s]                              

...

monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().


2024-11-09 04:42:41,974 - INFO - Downloaded: /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary2/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt
2024-11-09 04:42:41,974 - INFO - Expected md5 is None, skip md5 check for file /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary2/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt.


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
2024/11/09 04:42:42 INFO mlflow.

2024-11-09 07:09:22,438 - INFO - Ensembling using single GPU!
2024-11-09 07:09:22,439 - INFO - The output_dir is not specified. /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary2/ensemble_output will be used to save ensemble predictions.
2024-11-09 07:09:22,439 - INFO - Directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary2/ensemble_output is created to save ensemble predictions
2024-11-09 07:09:22,463 - INFO - Auto3Dseg picked the following networks to ensemble:
2024-11-09 07:09:22,464 - INFO - swinunetr_0
2024-11-09 07:09:22,464 - INFO - swinunetr_1
2024-11-09 07:09:22,464 - INFO - swinunetr_2
2024-11-09 07:09:22,464 - INFO - swinunetr_3
2024-11-09 07:09:22,465 - INFO - swinunetr_4
2024-11-09 07:09:22,465 - INFO - Auto3Dseg ensemble prediction outputs will be saved in /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary2/ensemble_output.


Ensembling (rank 0)...:   0%|          | 0/5 [00:00<?, ?it/s]monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `we

In [ ]:
scan_path = "/mnt/t/Data/3Tpioneer_bids/sub-ms1001/ses-20170215/proc/lesion_index.t3m20-mni_reg.nii.gz"
img = nib.load(scan_path)


In [38]:
dataroot

PosixPath('/mnt/h/3Tpioneer_bids')

In [36]:
import numpy as np
from monai.auto3dseg import FgImageStats

input = {}
input['image'] = nib.load(dataset[0].image).get_fdata()
input['image'] = np.moveaxis(input['image'], -1, 0)
input['label'] = nib.load(dataset[0].label).get_fdata()
analyzer = FgImageStats(image_key='image', label_key='label')
print(analyzer(input)["image_foreground_stats"])

{intensity: [{'max': 1.249400019645691, 'mean': 0.585796878314022, 'median': 0.5439000129699707, 'min': 0.08329999446868896, 'stdev': 0.18512559606395076, 'percentile': [0.2657249830663204, 0.38679999113082886, 0.8765999674797058, 1.1460999548435211], 'percentile_00_5': 0.2657249830663204, 'percentile_10_0': 0.38679999113082886, 'percentile_90_0': 0.8765999674797058, 'percentile_99_5': 1.1460999548435211}, {'max': 1760.9627685546875, 'mean': 697.8903051592728, 'median': 649.264404296875, 'min': 36.2552604675293, 'stdev': 298.28309908667416, 'percentile': [136.68988418579102, 327.8006134033203, 1130.0955810546875, 1399.7532043457031], 'percentile_00_5': 136.68988418579102, 'percentile_10_0': 327.8006134033203, 'percentile_90_0': 1130.0955810546875, 'percentile_99_5': 1399.7532043457031}]}


In [35]:
idx = [3,0,1,2]
np.moveaxis(input['image'], -1, 0).shape

(2, 240, 300, 300)

In [40]:
img = nib.load(dataset[0].image)

In [43]:
img.affine

array([[ 7.98769057e-01, -7.61604309e-03, -4.38079834e-02,
        -8.25058212e+01],
       [ 7.60465860e-03,  7.99963772e-01, -4.16040421e-04,
        -1.20926155e+02],
       [ 4.38102484e-02,  0.00000000e+00,  7.98799574e-01,
        -1.15306671e+02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [39]:
nib.save(input['image'], 'test.nii')

AttributeError: 'numpy.ndarray' object has no attribute 'to_filename'